# This notebook tests the function of the systemverilog hardware code generatoin function


In [3]:
import time
import os 
import numpy as np
import pandas as pd
import BelleII_CNN
import tensorflow.compat.v2 as tf                  # This TensorFlow Version is required for hls4ml
import matplotlib.pyplot as plt

2023-04-06 16:51:55.587848: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [4]:
(iy,ix) = (59,163)

In [5]:
kernel001 = np.array([
    [1,1,1,1,1,1,1],
    [1,2,2,2,2,2,1],
    [1,2,3,3,3,2,1],
    [1,2,3,4,3,2,1],
    [1,2,3,3,3,2,1],
    [1,2,2,2,2,2,1],
    [1,1,1,1,1,1,1]
])# Change the resolution here !!!
#(iy,ix) = (59,163)
# 




FPGA_parallel_conv2d_ins = BelleII_CNN.FPGA_parallel_conv2D(
# input shape   
input_size_x= ix,
input_size_y= iy,

# kernel param and strides
filter_param = kernel001,
filter_strides_x = 4 ,
filter_strides_y = 4,

# pool_type, size, strides
pool_type      = "average_pool",    
pool_size      = (2,2),
pool_strides   = (2,2),
# parallel input 
adder_input_size = 3
)

NEW SHIT JUST GOT MADE
The give input image shape is (59 , 163 ) 
Valid pix range under this conv-configuration (59,163 )
Convolution window range is (14,40)
-->Nice, All conv_pix in X direction is valid
-->Nice, All conv_pix on Y direction is Valid
adder tree solution with current setting found! 
Valid pix range under this pool-configuration ((14, 40))
 There are 20 valid pool window on X direction
 There are 7 valid pool window on Y direction


2023-04-06 16:52:21.963340: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-04-06 16:52:21.982627: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-04-06 16:52:22.054854: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.815GHz coreCount: 46 deviceMemorySize: 7.78GiB deviceMemoryBandwidth: 417.23GiB/s
2023-04-06 16:52:22.054913: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-04-06 16:52:26.272110: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2023-04-06 16:52:26.272207: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.1

In [6]:
dummy = FPGA_parallel_conv2d_ins.forward_calculation(np.ones(shape=(1,iy,ix,1)))
print(dummy.shape)
dummy_conv_window = FPGA_parallel_conv2d_ins._get_convwindow_position()
print(dummy_conv_window)
print(dummy_conv_window.shape)
dummy_pool_window = FPGA_parallel_conv2d_ins._get_poolwindow_position()
print(dummy_pool_window)
print(dummy_pool_window.shape)
dummy_addder_tree = FPGA_parallel_conv2d_ins._create_adder_configuration_matrix(3)
print(dummy_addder_tree)
print(dummy_addder_tree.shape)

(1, 7, 20, 1)
[[[0 0]
  [1 0]
  [2 0]
  ...
  [4 6]
  [5 6]
  [6 6]]

 [[4 0]
  [5 0]
  [6 0]
  ...
  [8 6]
  [9 6]
  [10 6]]

 [[8 0]
  [9 0]
  [10 0]
  ...
  [12 6]
  [13 6]
  [14 6]]

 ...

 [[148 52]
  [149 52]
  [150 52]
  ...
  [152 58]
  [153 58]
  [154 58]]

 [[152 52]
  [153 52]
  [154 52]
  ...
  [156 58]
  [157 58]
  [158 58]]

 [[156 52]
  [157 52]
  [158 52]
  ...
  [160 58]
  [161 58]
  [162 58]]]
(560, 49, 2)
[[[0 0]
  [1 0]
  [0 1]
  [1 1]]

 [[2 0]
  [3 0]
  [2 1]
  [3 1]]

 [[4 0]
  [5 0]
  [4 1]
  [5 1]]

 ...

 [[34 12]
  [35 12]
  [34 13]
  [35 13]]

 [[36 12]
  [37 12]
  [36 13]
  [37 13]]

 [[38 12]
  [39 12]
  [38 13]
  [39 13]]]
(140, 4, 2)
adder tree solution with current setting found! 
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48], [[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11], [12, 13, 14], [15, 16, 17], [18, 19, 20],

AttributeError: 'list' object has no attribute 'shape'

In [7]:
FPGA_parallel_conv2d_ins._get_poolwindow_position

<bound method FPGA_parallel_conv2D._get_poolwindow_position of <BelleII_CNN.FPGA_parallel_conv2D object at 0x7f11f39496a0>>

In [17]:
import numpy as np
import math

class FPGA_parallel_conv_pool():
    def __init__(
                self,
                input_shape,                     # defined as 2d tuple

                filter_param,                    # defined as 2d np array, fixed to int
                filter_stride,                   # defined as 2d tuple

                pool_type,                       # defined as string, either "max" or "average"
                pool_size,                       # defined as 2d tuple
                pool_stride,                     # defined as 2d tuple
            
                adder_input_size                 # defined as int, indicate how many parallel input for a adder
                ):
        
        # input data from preprocessing module
        self.input_x = input_shape[1]
        self.input_y = input_shape[0] 
        
        # Convolution Layer setting 
        self.filter_x = filter_param.shape[1]
        self.filter_y = filter_param.shape[0] 
        self.filter_param_np = filter_param 
        self.filter_param = tf.constant(filter_param.reshape(self.filter_y,self.filter_y,1,1),dtype=tf.float32)
        self.filter_stride_x = filter_stride[1]
        self.filter_stride_y = filter_stride[0] 
        self.filter_stride =[1,filter_stride[0],filter_stride[1],1] # input chanel and output channel are fixed to 1 
        self.adder_input_size = adder_input_size
        
        # Pool Layer
        self.pool_type = pool_type
        self.pool_size = pool_size
        self.pool_x    = pool_size[1]
        self.pool_y    = pool_size[0]
        self.pool_stride = pool_stride
        self.pool_stride_x = pool_stride[1]
        self.pool_stride_y = pool_stride[0]    

        # Conv_Window and Pool_Window 
        ## Conv Window
        self.conv_window_x = (self.input_x - self.filter_x) // self.filter_stride_x + 1
        self.conv_window_y = (self.input_y - self.filter_y) // self.filter_stride_y + 1
        self.conv_covered_pix_x = (self.conv_window_x - 1) * self.filter_stride_x + self.filter_x
        self.conv_covered_pix_y = (self.conv_window_y - 1) * self.filter_stride_y + self.filter_y
        ## Pool Window
        self.pool_window_x = (self.conv_window_x - self.pool_x) // self.pool_stride_x + 1 
        self.pool_window_y = (self.conv_window_y - self.pool_y) // self.pool_stride_y + 1
        self.pool_covered_conv_window_x = (self.pool_window_x - 1) * self.pool_stride_x + self.pool_x
        self.pool_covered_conv_window_y = (self.pool_window_y - 1) * self.pool_stride_y + self.pool_y

        # Get Conv_Window matrix, Pool_Window_matrix, Adder_tree_matrix
        self.conv_window_matrix = self._get_conv_window_matrix()                               # 3d np array with shape (Window_range, pix_in_window, position_of_pix )
        self.pool_window_matrix = self._get_pool_window_matrix()                               # 3d np array with shape (Window_range, pix_in_window, position_of_pix )
        self.adder_tree_connection_matrix  = self._get_connection_matrix(input_layer_size=self.filter_x*self.filter_y)     # 3d np array with shape (Stage_idx,adder_connections,parallel_input_num_to_adder)
        self.pool_connection_matrix        = self._get_connection_matrix(input_layer_size=self.pool_x * self.pool_y)
        # Print Setting 
        self.print_setting()

    def print_setting(self):
        """
            This Function checks the setting, if all pixs all covered by the convolution window or pool window
            This Function is supposed to be called by the __init__() function
        """
        #print(f"New Parallel conv pool layer generated")
        print(f"-------> Input Layer ({self.input_y},{self.input_x}) ")
        print(f"-> Conv Kernel ({self.filter_stride_y},{self.filter_stride_x})")
        print(f"-> Conv Stride ({self.filter_stride_y},{self.filter_stride_x})")
        print(f"-------> Conv Window ({self.conv_window_y},{self.conv_window_x})")
        print(f"-------> Covered pix ({self.conv_covered_pix_y},{self.conv_covered_pix_x})")
        print(f"-> Pool kernel ({self.pool_y},{self.pool_x})")
        print(f"-> Pool Stride ({self.pool_stride_y},{self.pool_stride_x})")
        print(f"-------> Pool Window ({self.pool_window_y},{self.pool_window_x})")
        print(f"-------> Covered win ({self.pool_covered_conv_window_y},{self.pool_covered_conv_window_x})")
        print(f"-------------> missed pix check")
        print(f"-------------> Conv missed ({self.input_y-self.conv_covered_pix_y},{self.input_x-self.conv_covered_pix_x}) pixels")
        print(f"-------------> Pool missed ({self.conv_window_y-self.pool_covered_conv_window_y},{self.conv_window_x-self.pool_covered_conv_window_x}) conv windows")
        #print(f"-------> Pool missed ({self.conv_covered_pix_y-self.pool_covered_pix_y},{self.conv_covered_pix_x-self.pool_covered_pix_x})")



    def _get_conv_window_matrix(self):
        """
            This Function returns the 3d conv_window_matrix [window_range,pix_in_window,x and y position]
        """
        conv_window_matrix = []   
        for window_num_y in range(self.conv_window_y):
            for window_num_x in range(self.conv_window_x):
                covered_pix_in_this_window = []

                # anker point defined as the left upper corner of each conv window
                anker_x,anker_y = window_num_x * self.filter_stride_x , window_num_y * self.filter_stride_y

                # calculate all the pixel in that window 
                for y_of_pixs_in_this_window in range(anker_y,anker_y + self.filter_y):
                    for x_of_pixs_in_this_window in range(anker_x,anker_x + self.filter_x):
                        covered_pix_in_this_window.append([x_of_pixs_in_this_window,y_of_pixs_in_this_window])

                # append the position [x,y] in the corresponding window
                conv_window_matrix.append(covered_pix_in_this_window)
        return np.array(conv_window_matrix)
    


    def _get_pool_window_matrix(self):
        """
            This Function returns the 3d pool_winodw_matrix [window_range,pix_in_window, x and y position]
        """
        pool_window_matrix = []
        for window_num_y in range(self.pool_window_y):
            for window_num_x in range(self.pool_window_x):
                covered_pix_in_this_window = []
                
                # anker point defined as the left upper corner of each pool window
                anker_x,anker_y = window_num_x * self.pool_stride_x, window_num_y * self.pool_stride_y

                #calculate all the pixels in that window
                for y_of_pixs_in_this_window in range(anker_y,anker_y + self.pool_stride_y):
                    for x_of_pixs_in_this_window in range(anker_x,anker_x + self.pool_stride_x):
                        covered_pix_in_this_window.append([x_of_pixs_in_this_window,y_of_pixs_in_this_window])
                pool_window_matrix.append(covered_pix_in_this_window)
        return np.array(pool_window_matrix)



    def _get_connection_matrix(self,input_layer_size):
        """
            This Function return the 3d connection matrix [Stage_idx, adder_connections, adder_parallel_input_size]
        """
        connection_matrix_per_stage = []
        connection_matrix_first_stage = [i for i in range(input_layer_size)]
        
        # First input stage 
        connection_matrix_per_stage.append(connection_matrix_first_stage)

        # Process the connection matrix stage by stage untill the outcome width is only 1 and then exit 
        process = True
        anker = 0
        stage_in_process = 0

        while(process):
            
            # households for each run
            input_size_this_stage = len(connection_matrix_per_stage[stage_in_process])
            connection_matrix_this_stage = []
            anker = 0

            # complete adder connections (all required parallel input of an adder are connected to a previous node)
            for piplined_reg in range(input_size_this_stage // self.adder_input_size):
                connections_to_this_piplined_reg = []
                for connections in range(piplined_reg * self.adder_input_size,(piplined_reg + 1) * self.adder_input_size):
                    connections_to_this_piplined_reg.append(connections)
                    anker = connections
                connection_matrix_this_stage.append(connections_to_this_piplined_reg)
            
            # incomplete  adder connections (some of adder input are not used)
            if(input_size_this_stage % self.adder_input_size != 0):
                connections_to_this_piplined_reg = []
                if(input_size_this_stage < self.adder_input_size):
                    for connections in range(input_size_this_stage % self.adder_input_size):
                        connections_to_this_piplined_reg.append(anker + connections)
                else:
                    for connections in range(input_size_this_stage % self.adder_input_size):
                        connections_to_this_piplined_reg.append(anker + connections + 1)
                connection_matrix_this_stage.append(connections_to_this_piplined_reg)

            connection_matrix_per_stage.append(connection_matrix_this_stage)
            stage_in_process += 1

            if len(connection_matrix_per_stage[stage_in_process]) == 1:
                print("hallo mother fucker")
                break
            if stage_in_process >= 99:
                raise ValueError(f"adder tree stage passed 99, failed to find a solution, pls check this shit!")

        return connection_matrix_per_stage
    
    def _get_pool_tree_connection_matrix(self):
        connection_matrix_per_stage = []
        connection_matrix_first_stage = [i for i in range()]
    
##############################################################################################################
######################################  Forward Propagation Methode  #########################################
##############################################################################################################
    """
    """

    def forward_propagation(self,x):
        conved_image = tf.nn.conv2d(input=x, filters=self.filter_param,strides=self.filter_stride,padding="VALID")
        if self.pool_type == "max":
            pooled_image = tf.nn.max_pool2d(input=conved_image, ksize=self.pool_size, strides=self.pool_stride,padding="VALID")
        elif self.pool_type == "average":
            pooled_image = tf.nn.avg_pool2d(input=conved_image, ksize=self.pool_size, strides=self.pool_stride,padding="VALID")
        else:
            raise TypeError(f"Pooling typ is unvalid, pls give either <max> or <average>")
        
        return (conved_image,pooled_image)
    
##############################################################################################################
###################################  System Verilog generation Methode  ######################################
############################################################################################################## 
    """
        _create_SV  Functions returns the SystemVerilog code for each submodule
        _create_INS Functions returns the Module instance for the top file
    """
##############################################  conv window  #################################################
    def _create_SV_conv_window_spliter(self):

        code = ""
        ## ports
        code += f"module conv2d_window_spliter(                                    \n  "
        code += f"input wire HIT_IMAGE_PIX[{self.input_y-1}:0][{self.input_y-1}:0], \n  "
        code += f"output wire [0:{self.filter_x * self.filter_x - 1}] CONV_WINDOW [0:{self.conv_window_x*self.conv_window_x-1}]\n"
        code += f");\n"

        ## rewiring
        for windowID,connected_pixs in enumerate(self.conv_window_matrix):
            code += f"assign CONV_WINDOW[{windowID}] = " + "{"
            for _,[pix_x,pix_y] in enumerate(connected_pixs):
                code += f"HIT_IMAGE_PIX[{pix_y}][{pix_x}]" if _== 0 else f", HIT_IMAGE_PIX[{pix_y}][{pix_x}]"
            code += "};\n"
        code += "endmodule"
        return code
    
    def _create_INS_conv_window_spliter(self):

        code = ""
        ## output connection wires
        code += f"wire [0:{self.filter_x * self.filter_x - 1}] CONV_WINDOW [0:{self.conv_window_x*self.conv_window_x-1}];\n"

        ## instance
        code += f"conv2d_windiw_spliter conv2d_windiw_spliter_ins (                 \n "
        code += f".HIT_IMAGE_PIX(HIT_IMAGE_PIX),                                    \n "
        code += f".CONV_WINDOW (CONV_WINDOW)                                      \n "
        code += f");"
        return code
##################################################  conv2d  ###################################################
    def _create_SV_adder_tree(self):

        ## Get settings for the adder tree implementation
        stage_connection_cfg = self.adder_tree_connection_matrix
        ## get kernel weights
        param_array    = self.filter_param_np.flatten() 
        max_param      = np.max(param_array)
        ## bit_width 
        total_adder_stage = len(stage_connection_cfg) - 1 # without the first input layer
        overflow_digits   = self.adder_input_size - 1
        bitwidth_param = int(math.log2(max_param)) + 1
        self.bitwidth_conv_result = bitwidth_param + total_adder_stage * overflow_digits

        ## ports
        code =  f""
        code += f"module conv2d_adder_tree(                                                \n "
        code += f"input wire CLK,                                                          \n "
        code += f"input wire  [0:{self.filter_stride_x*self.filter_stride_y}] PIX,         \n "
        code += f"output wire [{self.bitwidth_conv_result-1}:0] CONV_RESULT                     \n "
        code += f");\n"

        ## parameter 
        for idx, weight in enumerate(stage_connection_cfg[1:]):
            code += f"parameter W_{idx} = {weight};                                        \n "

        ## regs
        code += "// regs for each partial sum stage"
        for stage_idx,connections in enumerate(stage_connection_cfg[1:]):
            reg_bitwidth_for_this_stage = bitwidth_param + (stage_idx + 1) * overflow_digits # include the first input layer
            code += f"reg [{reg_bitwidth_for_this_stage-1}:0] S_{stage_idx} [{len(connections)-1}:0];\n" if stage_idx < total_adder_stage - 1\
            else    f"reg [{reg_bitwidth_for_this_stage-1}:0] S_{stage_idx}" 

        ## structure
        code += f"always@(posedge(CLK)) begin\n"
        ### Loop through all partial sum stages
        for stage_idx,stage_connections in enumerate(stage_connection_cfg[1:]):  
            code += f" // Partial Sum Stage{stage_idx}"
            ### Loop through all the adder node in that stage
            for node_idx, node_connections in enumerate(stage_connections):
                code += f"S_{stage_idx}[{node_idx}] <= \n " if stage_idx < total_adder_stage - 1\
                else    f"S_{stage_idx} <= "        # the final output stage
                new_line_flag = True
                ### Loop through all input(index by the previous node) to this adder node
                for _, node_from_previous_stage in enumerate(node_connections):
                    if stage_idx == 0:
                        code += f"W_{node_from_previous_stage} * PIX[{node_from_previous_stage}]" if new_line_flag else\
                                f"+ W_{node_from_previous_stage} * PIX[{node_from_previous_stage}]"  
                        new_line_flag = False
                    else:
                        code += f" S_{stage_idx-1}[{node_from_previous_stage}]" if new_line_flag else\
                                f" + S_{stage_idx-1}[{node_from_previous_stage}]"
                        new_line_flag = False
                code += f";\n"
        code += f"end\n"
        ## output assignment
        code += f"assign CONV_RESULT = S_{total_adder_stage-1};   \n"
        code += f"endmodule" 
        return code
    
    def _create_INS_adder_tree(self):
        code =  f""
        code += f" // connection wires  \n "
        code += f"wire [{self.bitwidth_conv_result-1}:0] CONV_RESULT [0:{self.filter_x*self.filter_y-1}]"
        code += f"genvar i    ;         \n "
        code += f"for(i = 0; i < {self.filter_x*self.filter_y} ; i = i + 1) begin      \n "
        code += f"conv2d_adder_tree conv2d_adder_tree_ins (                            \n"
        code += f".CLK(CLK),                                                           \n "
        code += f".PIX(CONV_WINDOW[i]),                                                \n "
        code += f".CONV_RESULT(CONV_RESULT[i])                                         \n "
        code += f" );                                                                  \n "
        code += f"end                                                                  \n "
        code += f"endgenerate                                                          \n "                
        return code
#################################################  pool window  ################################################    
    def _create_SV_pool_window_spliter(self):
        code = ""
        
        ##ports 
        code += f"module pool_window_spliter (           \n "
        code += f"input wire [{self.bitwidth_conv_result-1}:0] CONV_RESULT [0:{self.filter_x*self.filter_y-1}],    \n"
        code += f"output wire [0: {self.pool_x*self.pool_y-1}] [{self.bitwidth_conv_result-1}:0] POOL_WINDOW [0:{self.pool_window_x*self.pool_window_y-1}]    \n"
        code += f");    \n"

        ##rewiring
        for poolwindowID, connected_conv_window in enumerate(self.pool_window_matrix):
            code += f"assign POOL_WINDOW [{poolwindowID}] = " + "{"
            for _, [convwindow_x,convwindow_y] in enumerate(connected_conv_window):
                code += f"CONV_RESULT [ {convwindow_y*self.conv_window_x + convwindow_x}]" if _ == 0 else \
                        f",CONV_RESULT [ {convwindow_y*self.conv_window_x + convwindow_x}]"
            code += "}; \n"
        code += f"endmodule"
        return code
    
    def _create_INS_pool_window_spliter(self):
        code = ""
        ## wires
        code += f"wire [{self.bitwidth_conv_result-1}:0] CONV_RESULT [0:{self.filter_x*self.filter_y-1}], \n"
        ## port maps
        code += f"pool_window_spliter  pool_window_spliter_ins(                \n"
        code += f".CONV_RESULT(CONV_RESULT),                                    \n"
        code += f".POOL_WINDOW(POOL_WINDOW)                                      \n"
        code += f");                                                                \n"
        return code
    
##################################################  pool2d  ##################################################
    def _create_SV_pool2d(self):
        pass
    def _create_INS_pool2d(self):
        code = ""
        return code


    def _create_SV_max_pool(self):
        ## ports
        code =  f"module max_pool(                                                                                          \n"
        code += f"input wire CLK,                                                                                          \n"
        code += f"input wire  [0: {self.pool_x*self.pool_y-1}] [{self.bitwidth_conv_result-1}:0] POOL_WINDOW   ,            \n"
        code += f"output wire [{self.bitwidth_conv_result-1}:0]   POOL_RESULT                                          \n"
        code += f");   \n"
        ##

        return code 
    def _create_INS_max_pool(self):
        pass
    def _create_SV_average_pool(self):
        pass
    def _create_INS_average_pool(self):
        pass


In [28]:
dummy2 = FPGA_parallel_conv_pool(
input_shape=(iy,ix),
filter_param=kernel001,
filter_stride=(4,4),
pool_type="max",
pool_size=(2,2),
pool_stride=(2,2),
adder_input_size=2)
#print(dummy2.conv_window_matrix)
print(dummy2.conv_window_matrix.shape)
#print(dummy2.pool_window_matrix)
print(dummy2.pool_window_matrix.shape)
print(dummy2.adder_tree_connection_matrix)
print(dummy2.pool_connection_matrix)

hallo mother fucker
hallo mother fucker
-------> Input Layer (59,163) 
-> Conv Kernel (4,4)
-> Conv Stride (4,4)
-------> Conv Window (14,40)
-------> Covered pix (59,163)
-> Pool kernel (2,2)
-> Pool Stride (2,2)
-------> Pool Window (7,20)
-------> Covered win (14,40)
-------------> missed pix check
-------------> Conv missed (0,0) pixels
-------------> Pool missed (0,0) conv windows
(560, 49, 2)
(140, 4, 2)
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48], [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9], [10, 11], [12, 13], [14, 15], [16, 17], [18, 19], [20, 21], [22, 23], [24, 25], [26, 27], [28, 29], [30, 31], [32, 33], [34, 35], [36, 37], [38, 39], [40, 41], [42, 43], [44, 45], [46, 47], [48]], [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9], [10, 11], [12, 13], [14, 15], [16, 17], [18, 19], [20, 21], [22, 23], [24]], [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9], 